# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [25]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv', index_col=0)
# YOUR CODE HERE
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [26]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading']==True,:]
crypto_df.index = crypto_df.index.set_names([1])
crypto_df.shape

(1144, 6)

In [27]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
1,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [28]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(['IsTrading'], axis=1)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [29]:
# Remove rows that have at least 1 null value.
crypto_df_cleaned = crypto_df.dropna()
crypto_df_cleaned.shape
crypto_df_cleaned.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [30]:
# Keep the rows where coins are mined.
crypto_df_cleaned = crypto_df_cleaned.loc[crypto_df_cleaned['TotalCoinsMined']>0,:]
crypto_df_cleaned.shape

(532, 5)

In [31]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_df_cleaned_coin_name = crypto_df_cleaned[['CoinName']]
print(crypto_df_cleaned_coin_name.shape)
crypto_df_cleaned_coin_name.head()

(532, 1)


,CoinName
1,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [32]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df_cleaned = crypto_df_cleaned.drop(['CoinName'], axis=1)
print(crypto_df_cleaned.shape)
crypto_df_cleaned.head()

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [33]:
# Use get_dummies() to create variables for text features.
crypto_df_cleaned['TotalCoinSupply'] = crypto_df_cleaned['TotalCoinSupply'].astype('float64')
crypto_df_cleaned.dtypes
crypto_df_cleaned_dummies = pd.get_dummies(crypto_df_cleaned)
crypto_df_cleaned_dummies.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
1,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Standardize the data with StandardScaler().
crypto_df_scaled = StandardScaler().fit_transform(crypto_df_cleaned_dummies)
crypto_df_scaled
# YOUR CODE HERE

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [35]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
# Get two principal components for the iris data.
crypto_df_scaled_pca = pca.fit_transform(crypto_df_scaled)
crypto_df_scaled_pca

array([[-0.34263691,  1.01421571, -0.57669588],
       [-0.32589421,  1.01398815, -0.57700482],
       [ 2.32989479,  1.61462627, -0.73791994],
       ...,
       [ 0.28921079, -2.17088786,  0.37459382],
       [-0.22923256, -1.80739525,  0.35972426],
       [-0.26702172,  0.6725352 , -0.22906812]])

In [36]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame({'PC1':[each_val[0] for each_val in crypto_df_scaled_pca], 'PC2':[each_val[1] for each_val in crypto_df_scaled_pca], 'PC3':[each_val[2] for each_val in crypto_df_scaled_pca]},index=crypto_df_cleaned.index)
print(pcs_df.shape)
# YOUR CODE HERE
pcs_df.head(10)

(532, 3)


,PC1,PC2,PC3
1,,,
42,-0.342637,1.014216,-0.576696
404,-0.325894,1.013988,-0.577005
1337,2.329895,1.614626,-0.737920
BTC,-0.138484,-1.304444,0.214911
ETH,-0.170698,-1.999816,0.439010
LTC,-0.183636,-1.113987,0.008210
DASH,-0.378970,1.249551,-0.524875
XMR,-0.151888,-2.169377,0.519338
ETC,-0.169137,-1.999935,0.438990


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [37]:
# Create an elbow curve to find the best value for K.
# Looking for the best K
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
print(inertia)

[3636.695558299938, 2464.270011909228, 1509.4007405848995, 606.3802001064873, 426.6765073292538, 306.66155979116115, 246.77348509442888, 192.7672041990072, 155.06176185260236, 138.62611825334835]


In [38]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
print(df_elbow)


# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

    k      inertia
0   1  3636.695558
1   2  2464.270012
2   3  1509.400741
3   4   606.380200
4   5   426.676507
5   6   306.661560
6   7   246.773485
7   8   192.767204
8   9   155.061762
9  10   138.626118


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [39]:
# Initial imports
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.pandas

# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [44]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
clustered_df = pd.DataFrame()

clustered_df = crypto_df_cleaned
clustered_df['CoinName'] = crypto_df_cleaned_coin_name
clustered_df['Class'] = predictions
clustered_df['PC1'] = pcs_df['PC1']
clustered_df['PC2'] = pcs_df['PC2']
clustered_df['PC3'] = pcs_df['PC3']
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class,CoinName,PC1,PC2,PC3
1,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,0,42 Coin,-0.342637,1.014216,-0.576696
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,0,404Coin,-0.325894,1.013988,-0.577005
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,0,EliteCoin,2.329895,1.614626,-0.737920
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,1,Bitcoin,-0.138484,-1.304444,0.214911
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,1,Ethereum,-0.170698,-1.999816,0.439010
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,1,Litecoin,-0.183636,-1.113987,0.008210
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,0,Dash,-0.378970,1.249551,-0.524875
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,1,Monero,-0.151888,-2.169377,0.519338
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,1,Ethereum Classic,-0.169137,-1.999935,0.438990


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [48]:
# Creating a 3D-Scatter with the PCA data and the clusters
# clustered_df.
fig = px.scatter_3d(clustered_df, x="PC1", y="PC2", z="PC3", color="Class", symbol="Class",width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [56]:
# Create a table with tradable cryptocurrencies.
crypto_df_tradable = crypto_df.dropna()
crypto_df_tradable = crypto_df_tradable.loc[crypto_df_tradable['TotalCoinsMined']>0,:]

print(crypto_df_tradable.shape)
crypto_df_tradable.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
1,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [67]:
# Print the total number of tradable cryptocurrencies.
crypto_df_tradable= crypto_df_tradable[['TotalCoinSupply','TotalCoinsMined']]
crypto_df_tradable

,TotalCoinSupply,TotalCoinsMined
1,,
42,42,4.199995e+01
404,532000000,1.055185e+09
1337,314159265359,2.927942e+10
BTC,21000000,1.792718e+07
ETH,0,1.076842e+08
...,...,...
ZEPH,2000000000,2.000000e+09
GAP,250000000,1.493105e+07
BDX,1400222610,9.802226e+08


In [69]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
data_min_maxed = MinMaxScaler().fit_transform(crypto_df_tradable)
print(data_min_maxed)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [74]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
#TotalCoinSupply	TotalCoinsMined	CoinName	Class

plot_df = pd.DataFrame({'TotalCoinSupply':[data[0] for data in data_min_maxed],'TotalCoinsMined':[data[1] for data in data_min_maxed]}, index=crypto_df_tradable.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
1,,,,
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1


In [76]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class")

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)